# ============================================
# Arnold's Cat Map Project
# ============================================

# Developed by: Marouf Haider & Bahidj Nafaa
# School: National Higher School of Mathematics
# Course: Algebra & Coding (3rd Year)
# Academic Year: 2023–2024

# Imports

In [1]:
import numpy as np
import cv2
from numpy import ones as o
from numpy import lcm as l

# ENCODING USING ARNOLD'S CAT MAP

In [2]:
def apply_arnold_cat_map(image, iterations):
    
    # Get image size (assumes square image)
    N = image.shape[0]

    # Initialize empty image of same shape
    encoded_image = np.zeros_like(image)

    # Iterate through each pixel coordinate
    for x in range(N):
        for y in range(N):
            # Start with current pixel position
            x_new, y_new = x, y

            # Apply ACM transformation 'iterations' times
            for i in range(iterations):
                x_new, y_new = (x_new + y_new) % N, (x_new + 2 * y_new) % N

            # Place pixel in new position
            encoded_image[x_new, y_new] = image[x, y]

    # Return the encoded image
    return encoded_image


# MINIMAL PERIOD CALCULATION

In [3]:
def minimal_period(n):
    
    # Initialize least common multiple accumulator
    p = 1

    # Create n x n matrix of ones (to mark unvisited positions)
    li = o([n, n])

    # Iterate through every pixel (i, j)
    for i in range(n):
        for j in range(n):

            # If the current position has not been visited
            if int(li[i][j]) == 1:

                # Count steps (c) in the orbit of (i, j)
                c = 1
                i_1 = i
                j_1 = j

                # Apply the ACM repeatedly until we return to the original pixel
                while ((2 * i_1 + j_1) % n) != i or ((i_1 + j_1) % n) != j:
                    a = i_1
                    i_1 = (2 * i_1 + j_1) % n
                    j_1 = (a + j_1) % n
                    c += 1

                    # Mark this position as visited
                    li[i_1][j_1] = 0

                # Update the least common multiple with the orbit length
                p = l(p, c)

    return p


# RETRIEVING ORIGINAL IMAGE


In [4]:
def retrieve(encoded_image, iterations):     
    # Extract image size from encoded image (assumes global variable or previously loaded)
    N = encoded_image.shape[0]

    # Compute minimal period for this image size
    period = minimal_period(N)

    # Compute how many more iterations are needed to restore the image
    reverse_iterations = period - (iterations % period)

    # Apply inverse transformation using the remaining iterations
    return apply_arnold_cat_map(encoded_image, reverse_iterations)


# Example Usage

In [22]:
# Example: Encode an image using Arnold's Cat Map and show intermediate steps
input_image_path = 'assets/test_image.jpg'  # Path to square grayscale image in the assets folder
# Load image in grayscale mode
image = cv2.imread(input_image_path, cv2.IMREAD_GRAYSCALE)

# Check if image was loaded successfully
if image is None:
    raise ValueError("Image could not be read. Check the file path.")
 
# Check if image is square
if image.shape[0] != image.shape[1]:
    raise ValueError("Image must be square (same width and height).")   

iterations = 10  # Number of iterations for scrambling
current_image = image
for i in range(1, iterations + 1):
    current_image = apply_arnold_cat_map(current_image, 1)
    cv2.imshow('Encoding Progress', current_image)
    if i == iterations:
        cv2.waitKey(0)  # Wait for key press at the final result
    else:
        cv2.waitKey(500)  # Show intermediate steps for 0.5 seconds
cv2.destroyAllWindows()
encoded_image = current_image
cv2.imwrite(f'outputs/encoded_image{iterations}.jpg', encoded_image)

True

In [23]:
# Example: Retrieve the original image from the encoded image
iterations = 10  # The same number of iterations used for encoding
encoded_image_path = f'outputs/encoded_image{iterations}.jpg'  # Path to the scrambled image in the outputs folder

# Load image in grayscale mode
encoded_image = cv2.imread(encoded_image_path, cv2.IMREAD_GRAYSCALE)

# Check if image was loaded successfully
if encoded_image is None:
    raise ValueError("Image could not be read. Check the file path.")
 
# Check if image is square
if encoded_image.shape[0] != encoded_image.shape[1]:
    raise ValueError("Image must be square (same width and height).") 

# Retrieve the original image using the retrieve function
retrieved_image = retrieve(encoded_image, iterations)
cv2.imshow('Retrieved Image', retrieved_image)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imwrite('outputs/retrieved_image.jpg', retrieved_image)

True